## About Project

## Taking multiple FAQ questions and Building a chatbot for that with multiple data sources inclduing the homepage also
## Takinng out the questions and making it into the CSV formate for laoding
## Creating  a well documented files

In [1]:
from langchain_community.document_loaders import RecursiveUrlLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import HypotheticalDocumentEmbedder
from langchain_chroma import Chroma
import pandas as pd
import os

In [2]:
os.environ['HUGGINGFACEHUB_API_TOKEN']= ""

In [3]:
hf_embeddings = HuggingFaceEndpointEmbeddings(
    model= "sentence-transformers/all-MiniLM-L6-v2",
    task="feature-extraction",
    huggingfacehub_api_token="",
)

Llama3LLM = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_tokens=1000,
    do_sample=False,
)
Llama3LLMHydeEmbeddings = HypotheticalDocumentEmbedder.from_llm(Llama3LLM,
                                                  hf_embeddings,
                                                  prompt_key = "web_search")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\91845\.cache\huggingface\token
Login successful


In [4]:
loader = PyPDFDirectoryLoader(r"crime data\murder")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
documents=text_splitter.split_documents(docs)



In [5]:
## Design ChatPrompt Template
prompt = ChatPromptTemplate.from_template("""
just give main Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")


In [6]:
# Llama3LLMDB=Chroma.from_documents(documents[:10000],Llama3LLMHydeEmbeddings, persist_directory="./chroma")
Llama3LLMDB = Chroma(persist_directory="./chroma", embedding_function=Llama3LLMHydeEmbeddings)
Llama3LLMRetriever = Llama3LLMDB.as_retriever(type="similarity")

In [7]:
DocumentChainMistralLLM = create_stuff_documents_chain(Llama3LLM,prompt)
RetrievalChainMistralLLM = create_retrieval_chain(Llama3LLMRetriever,DocumentChainMistralLLM)

In [8]:
if __name__ == "__main__":
    MistralOutput = RetrievalChainMistralLLM.invoke({"input":"List of all the laws that are involed murder cases"})

In [10]:
print(MistralOutput['answer'])

 in this context.

Answer: The laws involved in this context, based on the provided context, are:

1. Law of Murder: This law deals with the offense of murder, which is the unlawful killing of another human being with malice aforethought.

2. Right of Private Defence: This law allows a person to use reasonable force to defend himself or his property from an unlawful attack. The force used should not be unduly disproportionate to the injury being averted or reasonably apprehended.

3. Provocation: This is a legal doctrine that can reduce a charge of murder to manslaughter, if it can be shown that the accused acted under extreme emotional duress. However, the provided context does not seem to discuss this in detail.

4. Law of Injuries: This law deals with the consequences when a person inflicts injuries of a kind that could potentially lead to death. Even if death is not the intended outcome, the person must still face the consequences. However, if it can be reasonably deduced that the 